In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.impute import SimpleImputer
from xverse.transformer import WOE

In [2]:
df = pd.read_csv('../data/data.csv')

In [3]:
df_fe = df.copy()

In [4]:
df_fe.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [5]:
df_fe.columns

Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')

Create Aggregate Features

In [6]:
agg_feature = df_fe.groupby('CustomerId')['Amount'].agg(['sum','mean','count','std','min','max']).reset_index()
agg_feature.columns = ['CustomerId','Amount_sum','Amount_mean','Amount_count','Amount_std','Amount_min','Amount_max']
agg_feature.head()

,CustomerId,Amount_sum,Amount_mean,Amount_count,Amount_std,Amount_min,Amount_max
0,CustomerId_1,-10000.0,-10000.000000,1,NaN,-10000.0,-10000.0
1,CustomerId_10,-10000.0,-10000.000000,1,NaN,-10000.0,-10000.0
2,CustomerId_1001,20000.0,4000.000000,5,6558.963333,-5000.0,10000.0
3,CustomerId_1002,4225.0,384.090909,11,560.498966,-75.0,1500.0
4,CustomerId_1003,20000.0,3333.333333,6,6030.478146,-5000.0,10000.0


In [7]:

df_fe = df_fe.merge(agg_feature, on='CustomerId', how='left')


In [8]:
agg_feature.tail()

,CustomerId,Amount_sum,Amount_mean,Amount_count,Amount_std,Amount_min,Amount_max
3737,CustomerId_992,20000.0,3333.333333,6,6088.240030,-5000.0,10000.0
3738,CustomerId_993,20000.0,4000.000000,5,6745.368782,-5000.0,10000.0
3739,CustomerId_994,543873.0,5384.881188,101,14800.656784,-5000.0,90000.0
3740,CustomerId_996,139000.0,8176.470588,17,4433.329648,-5000.0,10000.0
3741,CustomerId_998,100000.0,4545.454545,22,6775.145563,-5000.0,10000.0


Extract Features

In [9]:
df_fe['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])


In [10]:
df_fe['TransactionHour'] = df_fe['TransactionStartTime'].dt.hour
df_fe['TransactionDay'] = df_fe['TransactionStartTime'].dt.day
df_fe['TransactionMonth'] = df_fe['TransactionStartTime'].dt.month
df_fe['TransactionYear'] = df_fe['TransactionStartTime'].dt.year

In [11]:
df_fe.head(20)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,...,Amount_sum,Amount_mean,Amount_count,Amount_std,Amount_min,Amount_max,TransactionHour,TransactionDay,TransactionMonth,TransactionYear
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,...,109921.75,923.712185,119,3042.294251,-5000.0,20000.0,2,15,11,2018
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,...,109921.75,923.712185,119,3042.294251,-5000.0,20000.0,2,15,11,2018
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,...,1000.00,500.000000,2,0.000000,500.0,500.0,2,15,11,2018
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,...,228727.20,6019.136842,38,17169.241610,-10000.0,100000.0,3,15,11,2018
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,...,228727.20,6019.136842,38,17169.241610,-10000.0,100000.0,3,15,11,2018
5,TransactionId_23223,BatchId_25954,AccountId_1078,SubscriptionId_4238,CustomerId_1432,UGX,256,ProviderId_6,ProductId_3,airtime,...,2000.00,2000.000000,1,NaN,2000.0,2000.0,3,15,11,2018
6,TransactionId_118063,BatchId_118460,AccountId_2442,SubscriptionId_1980,CustomerId_2858,UGX,256,ProviderId_5,ProductId_3,airtime,...,93400.00,3220.689655,29,5493.966126,-5000.0,20000.0,3,15,11,2018
7,TransactionId_100640,BatchId_38561,AccountId_4841,SubscriptionId_3829,CustomerId_2858,UGX,256,ProviderId_4,ProductId_6,financial_services,...,93400.00,3220.689655,29,5493.966126,-5000.0,20000.0,3,15,11,2018
8,TransactionId_51905,BatchId_93774,AccountId_272,SubscriptionId_4731,CustomerId_598,UGX,256,ProviderId_6,ProductId_10,airtime,...,8060.00,2015.000000,4,3342.588817,-140.0,7000.0,4,15,11,2018
9,TransactionId_130161,BatchId_82409,AccountId_710,SubscriptionId_920,CustomerId_1053,UGX,256,ProviderId_1,ProductId_15,financial_services,...,13788.00,1969.714286,7,1911.444629,-12.0,4000.0,4,15,11,2018


In [20]:
target_col = 'FraudResult'
y = df_fe[target_col]
x = df_fe.drop(columns=[target_col,'TransactionId','BatchId','AccountId','SubscriptionId','TransactionStartTime'])

In [21]:
x.head()

,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,PricingStrategy,Amount_sum,Amount_mean,Amount_count,Amount_std,Amount_min,Amount_max,TransactionHour,TransactionDay,TransactionMonth,TransactionYear
0,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2,109921.75,923.712185,119,3042.294251,-5000.0,20000.0,2,15,11,2018
1,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2,109921.75,923.712185,119,3042.294251,-5000.0,20000.0,2,15,11,2018
2,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2,1000.00,500.000000,2,0.000000,500.0,500.0,2,15,11,2018
3,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2,228727.20,6019.136842,38,17169.241610,-10000.0,100000.0,3,15,11,2018
4,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2,228727.20,6019.136842,38,17169.241610,-10000.0,100000.0,3,15,11,2018


Encode Categorical Variables: Apply One-Hot Encoding 


In [22]:
cat_cols = [  'CurrencyCode','CountryCode','ProviderId','ProductId','ProductCategory','ChannelId','PricingStrategy']
num_cols = list(agg_feature.columns)

In [23]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('encoder', OneHotEncoder(
        handle_unknown='ignore',
        min_frequency=0.01
    ))
])


Standardize Numerical Features

In [24]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])


In [25]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ]
)


In [26]:
customer_df = agg_feature.shape
customer_df

(3742, 7)

In [27]:
x_processed = preprocessor.fit_transform(x)
print("preprocessed feature matrix shape:", x_processed.shape)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'CustomerId_4406'

In [ ]:
x_final = pd.DataFrame(x_processed, 
    columns=num_cols, 
    index=x.index  
)

Save the processed data 

In [ ]:
features_path = '../data/processed/customer_features.csv'
x_final.to_csv(features_path)

WoE and IV using xverse

In [ ]:
x_woe_input = x[cat_cols]

woe = WOE()
woe.fit(x_woe_input, y)


In [ ]:
print("WOE values")
print(woe.woe_df.head())

In [ ]:
print("information value(IV)")
print(woe.iv_df)

In [ ]:
x_cat_woe = woe.transform(x_woe_input,y)
print("woe transformed shape:",x_cat_woe.shape)